In [ ]:
# default_exp services/linode

In [ ]:
#export
import os
import sys
import pathlib
from pathlib import Path
import pprint
from linode_api4 import LinodeClient
import fire


class LinodeAPI(object):
    token: str = None
    verbose = 1
    save_path: Path = None
    tags = None
    allow_all = False
    
    def __init__(self, save_path=None, verbose=1, tags="ansible", all=None):
        if isinstance(verbose, int):
            self.verbose = verbose
        if save_path is not None:
            self.save_path = pathlib.Path(save_path)
        self.allow_all = all is not None
        if tags is not None:
            self.tags = tags

    def get_lookup_tags(self):
        if isinstance(self.tags, str):
            return [self.tags]
        if isinstance(self.tags, tuple) or isinstance(self.tags, list):
            return list(set(self.tags))
        return []

    def inventory_by_tag(self):
        """
        Compile Linode Inventory
        into dictionary groups
        """
        token = os.environ.get("LINODE_ACCESS_TOKEN")
        if not token:
            raise Exception(f'LINODE_ACCESS_TOKEN not in environment variables.')
        self.token = token
        tagged_inventory = {}
        client =  LinodeClient(self.token)
        instances = client.linode.instances()
        for current_linode in instances:
            ip_address = current_linode.ipv4
            linode_tags = current_linode.tags
            lookup_tags = self.get_lookup_tags()
            allow_all = self.allow_all
            if not allow_all and len(lookup_tags) > 0:
                tag_set = set(lookup_tags)
                linode_tags_set = set(linode_tags)
                intersection = linode_tags_set.intersection(tag_set)
                if len(tag_set) != 0:
                    if len(linode_tags_set) == 0:
                        continue
                    if not intersection:
                        continue
            if len(linode_tags) == 0:
                if isinstance(tagged_inventory.get('_ungrouped'), list):
                    tagged_inventory['_ungrouped'] += ip_address
                else:
                    tagged_inventory['_ungrouped'] =  ip_address
            else:
                for tag in linode_tags:
                    if isinstance(tagged_inventory.get(tag), list):
                        tagged_inventory[tag] += ip_address
                    else:
                        tagged_inventory[tag] =  ip_address
        return tagged_inventory
    
    def inventory(self):
        tagged_inventory = self.inventory_by_tag()
        invetory_doc_items = []
        lookup_tags = self.get_lookup_tags()
        allow_all = self.allow_all
        for key, val in tagged_inventory.items():
            if not isinstance(val, list):
                continue
            if not allow_all:
                if key not in lookup_tags:
                    continue
            if key == "_ungrouped":
                key = ""
                items = "\n".join(val) + "\n\n"
                invetory_doc_items.insert(0, items)
            else:
                group_name = f"[{key}]\n"
                items = "\n".join(val)
                group_str = group_name + items + '\n\n'
                invetory_doc_items.append(group_str)
        output = "".join(invetory_doc_items)
        if self.verbose == 1:
            sys.stdout.write(output)
        if self.save_path is not None:
            try:
                saved = self.save_path.write_text(output)
            except Exception as e:
                sys.stdout.write(f"Cannot save file to \"{self.save_path}\"\n\n{e}\n")